In [1]:
import streamlit as st
from streamlit_chat import message
from timeit import default_timer as timer

from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
import openai
from langchain.chat_models import ChatOpenAI

import dotenv
import os

dotenv.load_dotenv()

2024-12-18 15:49:26.309 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


True

In [2]:
llm = ChatOpenAI(
    model = "gpt-4",
    temperature=0,
    openai_api_key = openai.api_key
)


#Neo4j configuration
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")

# Cypher generation prompt
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Client, use `toLower(client.id) contains 'neo4j'`. To search for Slack Messages, use 'toLower(SlackMessage.text) contains 'neo4j'`. To search for a project, use `toLower(project.summary) contains 'logistics platform' OR toLower(project.name) contains 'logistics platform'`.)
5. Never use relationships that are not mentioned in the given schema
6. When asked about projects, Match the properties using case-insensitive matching and the OR-operator, E.g, to find a logistics platform -project, use `toLower(project.summary) contains 'logistics platform' OR toLower(project.name) contains 'logistics platform'`.

schema: {schema}

Examples:
Question: Which client's projects use most of our people?
Answer: ```MATCH (c:CLIENT)<-[:HAS_CLIENT]-(p:Project)-[:HAS_PEOPLE]->(person:Person)
RETURN c.name AS Client, COUNT(DISTINCT person) AS NumberOfPeople
ORDER BY NumberOfPeople DESC```
Question: Which person uses the largest number of different technologies?
Answer: ```MATCH (person:Person)-[:USES_TECH]->(tech:Technology)
RETURN person.name AS PersonName, COUNT(DISTINCT tech) AS NumberOfTechnologies
ORDER BY NumberOfTechnologies DESC```

Question: {question}
"""

cypher_prompt = PromptTemplate(
    template = cypher_generation_template,
    input_variables = ["schema", "question"]
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}

Question: {question}
Helpful Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)


/tmp/ipykernel_1405525/2796663808.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [5]:
def query_graph(user_input):
    graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password)
    chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        cypher_prompt=cypher_prompt,
        qa_prompt=qa_prompt,
        allow_dangerous_requests=True
        )
    result = chain(user_input)
    return result


In [8]:
a = query_graph("Which of our projects are using Azure")
print(a)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Project)-[:USES_TECH]->(t:Technology)
WHERE toLower(t.name) contains 'azure'
RETURN p.name AS ProjectName
Full Context:
[{'ProjectName': 'BetaHealth Telemedicine Platform on Microsoft Azure'}, {'ProjectName': 'BetaHealth Telemedicine Platform on Microsoft Azure'}, {'ProjectName': 'BetaHealth Telemedicine Platform on Microsoft Azure'}, {'ProjectName': 'BetaHealth Telemedicine Platform on Microsoft Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}, {'ProjectName': 'BetaHealth Secure Healthcare Data Analytics Platform on Azure'}]

> Finished chain.
{'query'

In [7]:
a

{'query': 'who is on charge of Azure project ? ',
 'result': "I'm sorry, but I don't have the information about who is in charge of the Azure project.",
 'intermediate_steps': [{'query': "MATCH (p:Project)<-[:HAS_PEOPLE]-(person:Person) \nWHERE toLower(p.name) contains 'azure' OR toLower(p.summary) contains 'azure'\nRETURN person.name AS PersonName"},
  {'context': [{'PersonName': 'Noah Wilson'},
    {'PersonName': 'Ava Jackson'},
    {'PersonName': 'Sarah Johnson'},
    {'PersonName': 'Amanda Rodriguez'},
    {'PersonName': 'Jessica White'},
    {'PersonName': 'William Lee'},
    {'PersonName': 'Lucas Taylor'},
    {'PersonName': 'Liam Thompson'},
    {'PersonName': 'Sophia Anderson'},
    {'PersonName': 'Lucas Taylor'}]}]}